<a href="https://colab.research.google.com/github/Zabiullahkhan/Data_Science/blob/main/DistilBert_77.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install datasets
!pip install transformers
# !pip install transformers[torch]
# !pip install accelerate -U
import transformers

print(transformers.__version__)

4.33.2


In [6]:
import os
import torch
import pandas as pd
import transformers
import tensorflow as tf
from sklearn.model_selection import train_test_split
from transformers import pipeline
from transformers import TextClassificationPipeline
from transformers import TFTrainingArguments, TFTrainer
from torch.utils.data import Dataset
from tqdm import tqdm

In [7]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
device

'cpu'

In [8]:
# !wget https://raw.githubusercontent.com/PolyAI-LDN/task-specific-datasets/master/banking_data/train.csv
# !wget https://raw.githubusercontent.com/PolyAI-LDN/task-specific-datasets/master/banking_data/test.csv
# !wget https://github.com/PolyAI-LDN/task-specific-datasets/blob/master/banking_data/categories.json

In [9]:
train_data = pd.read_csv('../content/train.csv')
test_data = pd.read_csv('../content/test.csv')
df = pd.concat([train_data, test_data], axis=0)
df = df.reset_index(drop=True)
df['label'] = df['category'].astype('category').cat.codes
id2label = {label:category for category, label in sorted(df[['category', 'label']].drop_duplicates().values, key = lambda x:x[1])}
label2id = {category:label for category, label in sorted(df[['category', 'label']].drop_duplicates().values, key = lambda x:x[1])}

NUM_LABELS = len(df['label'].drop_duplicates())
NUM_LABELS

77

In [10]:
df.head(2)

,text,category,label
0,I am still waiting on my card?,card_arrival,12
1,What can I do if my card still hasn't arrived ...,card_arrival,12


In [11]:
data_texts = df['text'].to_list()
data_labels = df['label'].to_list()


train_texts, val_texts, train_labels, val_labels = train_test_split(data_texts, data_labels, test_size= 0.2)
train_texts, test_texts, train_labels, test_labels = train_test_split(train_texts, train_labels, test_size = 0.1)

In [12]:
# from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
# from transformers import AutoTokenizer

# tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased", max_length = 512)


# model = AutoModelForSequenceClassification.from_pretrained(
#     "distilbert-base-uncased", num_labels=NUM_LABELS, id2label=id2label, label2id=label2id
# )
from transformers import DistilBertTokenizer, TFDistilBertForSequenceClassification

tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased', max_length = 512)
model = TFDistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=NUM_LABELS, id2label=id2label, label2id=label2id
)


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_transform.weight', 'vocab_transform.bias']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should 

In [13]:
# from transformers import RobertaTokenizer, TFRobertaModel
# tokenizer = RobertaTokenizer.from_pretrained('roberta-large', max_length = 512)
# model = TFRobertaModel.from_pretrained('roberta-large', num_labels = NUM_LABELS)
train_encodings = tokenizer(train_texts, truncation = True, padding = True)
val_encodings = tokenizer(val_texts, truncation = True, padding = True)

In [ ]:
# from transformers import RobertaTokenizer, TFRobertaModel
# tokenizer = RobertaTokenizer.from_pretrained('roberta-large', max_length = 512)
# model = TFRobertaModel.from_pretrained('roberta-large', num_labels = NUM_LABELS)
# train_encodings = tokenizer(train_texts, truncation = True, padding = True)
# val_encodings = tokenizer(val_texts, truncation = True, padding = True)

In [11]:
train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    train_labels
))

val_dataset = tf.data.Dataset.from_tensor_slices((
    dict(val_encodings),
    val_labels
))

In [12]:
training_args = TFTrainingArguments(
    output_dir = './results',
    num_train_epochs = 1,
    per_device_train_batch_size = 16,
    per_device_eval_batch_size = 16,
    warmup_steps = 500,
    weight_decay = 1e-5,
    logging_dir = './logs',
    eval_steps = 100

)


In [13]:
with training_args.strategy.scope():
  model = TFDistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=NUM_LABELS, id2label=id2label, label2id=label2id
)
trainer = TFTrainer(
    model = model,
    args = training_args,
    train_dataset = train_dataset,
    eval_dataset = val_dataset,
)


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_transform.bias', 'vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.bias']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should 

In [14]:
trainer.train()
trainer.evaluate()

{'eval_loss': 1.0750116487828696}

In [15]:

save_directory = "/save_models"

model.save_pretrained(save_directory)

tokenizer.save_pretrained(save_directory)

('/save_models/tokenizer_config.json',
 '/save_models/special_tokens_map.json',
 '/save_models/vocab.txt',
 '/save_models/added_tokens.json')

In [16]:
tokenizer_fine_tuned = DistilBertTokenizer.from_pretrained(save_directory)
model_fine_tuned = TFDistilBertForSequenceClassification.from_pretrained(save_directory)

Some layers from the model checkpoint at /save_models were not used when initializing TFDistilBertForSequenceClassification: ['dropout_39']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at /save_models and are newly initialized: ['dropout_59']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [17]:
test_text = test_texts[0]
test_text

'Over the holiday I withdrew cash from the ATM and it seems that I was severely overcharged. Had I known that I would be charged those fees, I would not have used the ATM!'

In [21]:
predict_input = tokenizer_fine_tuned.encode(
    test_text,
    truncation = True,
    padding = True,
    return_tensors = 'tf'
)

output = model_fine_tuned(predict_input)[0]
prediction_value = tf.argmax(output, axis = 1).numpy()[0]
print(prediction_value)
predicted_label = id2label[prediction_value]
predicted_label

20


'cash_withdrawal_charge'

In [22]:

predicted_label

'cash_withdrawal_charge'

**Inferencing With `Pytorch`**

In [14]:
import torch
from transformers import DistilBertForSequenceClassification, DistilBertTokenizer
save_directory = "/save_models"
tokenizer_fine_tuned_pt = DistilBertTokenizer.from_pretrained(save_directory)
model_fine_tuned_pt = DistilBertForSequenceClassification.from_pretrained(save_directory, from_tf = True)




All TF 2.0 model weights were used when initializing DistilBertForSequenceClassification.

All the weights of DistilBertForSequenceClassification were initialized from the TF 2.0 model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use DistilBertForSequenceClassification for predictions without further training.


In [ ]:
test_text = test_texts[0]
inputs = tokenizer_fine_tuned_pt(test_texts, return_tensors="pt", padding=True, truncation=True, max_length=512)
output = model_fine_tuned_pt(**inputs)

predicted_class = torch.argmax(output.logits, dim=2)[0].tolist()
predicted_class
# Decode the predicted class
# predicted_tokens = tokenizer_fine_tuned_pt.convert_ids_to_tokens(predicted_class)



# # Get the predicted class
# predicted_class = torch.argmax(output.logits, dim=1).tolist()[0]

# print("Predicted Class:", predicted_class)
